In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'bybit'
user = 'your_user_na me'
settings = load_settings(exchange, user)
s = 'BTCUSD'
n_days = 21

In [ ]:
#load cache if any
#agg_trades = pd.read_csv('btcusdt_agg_trades_bybit_7_days_2021-01-05.csv').set_index('trade_id')
#adf = agg_trades

In [ ]:
#otherwise
#agg_trades = await load_trades(exchange, user, s, n_days)

In [ ]:
# look at jackrabbit results
rdf = pd.concat([pd.read_csv('jackrabbit_results_grid/2021-01-06T13:02:45.csv')])
column_names = ['ddown_factor', 'grid_spacing', 'grid_spacing_coefficient',
               'initial_equity', 'markup']
rdf.columns = column_names + list(rdf.columns[len(column_names):])
#rdfs = rdf.sort_values('pnl_sum', ascending=[False])
rdfs = rdf.sort_values('gain', ascending=[False])
rdfs = rdfs[~rdfs.gain.duplicated()]
rdfs = rdfs[rdfs.n_closes > 0].head(30)
rdfs

In [ ]:
# set best jackrabbit result as settings
bs = dict(rdfs.iloc[0])
for k in bs:
    if k in settings:
        settings[k] = round(float(float(bs[k])), 10)
        print(k, settings[k])
settings = sort_dict_keys(settings)
settings

In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ll = longs[longs.type == 'liq']
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        longs.close_price.plot(style='r--')
        longs.liq_price.plot(style='k--')
        lc.price.plot(style='ro')
        ll.price.plot(style='gx')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        sl = shrts[shrts.type == 'liq']
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        shrts.close_price.plot(style='b--')
        shrts.liq_price.plot(style='k--')
        sc.price.plot(style='bo')
        sl.price.plot(style='gx')

In [ ]:
# prep df for backtesting
df = prep_df(adf)
print(len(df), len(df) / len(adf))
df.price.iloc[::100].plot()

In [ ]:
settings = {
    'compounding': False,
    'ddown_factor': 0.01,
    'grid_spacing': 0.003,
    'grid_spacing_coefficient': 20.0,
    'initial_equity': 0.001,
    'isolated_mode': False,
    'leverage': 100.0,
    'liq_modifier': 0.001,
    'maker_fee': -0.00025,
    'markup': 0.0019,
    'min_qty': 1.0,
    'n_days': 0,
    'price_step': 0.5,
    'qty_step': 1.0,
    'symbol': 'BTCUSD'
}
price_ = df.iloc[0].price
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
settings['n_days'] = n_days
print('price', price_)
print('approx price_step', price_ * settings['grid_spacing'])
settings

In [ ]:
# dump settings
#json.dump(settings, open(f'settings/bybit/{user}.json', 'w'), indent=4, sort_keys=True)

In [ ]:
# backtest
start_time = time()
trades = backtest(df, settings)
print('time elapsed', time() - start_time)

In [ ]:
# analyze results
tdf = pd.DataFrame(trades).set_index('trade_id')
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
ll = longs[longs.type == 'liq']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']
sl = shrts[shrts.type == 'liq']

margin_max = tdf.margin_cost.max()
pnl_sum = tdf.pnl.sum()
gain = (margin_max + pnl_sum) / margin_max
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
average_daily_gain = gain ** (1 / n_days)
liqs = tdf[tdf.type == 'liq']
closes = tdf[tdf.type == 'close']
print('margin_max', margin_max)
print('biggest qty', tdf.qty.abs().max())
print('pnl_sum', pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average_daily_gain', average_daily_gain)
print('n trades', len(tdf))
print('n closes', len(closes))
print('n liqs', len(liqs))

In [ ]:
# visualize behavior
step = 200
i = -step

In [ ]:
i += step
plot_tdf_(df, tdf.iloc[i:i+step])

In [ ]:
tdf.iloc[i:i+step].tail(40)

In [ ]:
(tdf.qty / tdf.equity).plot()

In [ ]:
tdf.equity_available.plot()

In [ ]:
# cumulative pnl
tdf.pnl.cumsum().plot()

In [ ]:
# liquidations
liqs

In [ ]:
# inspect liquidations
liq_i = 0
liq_iloc = tdf.index.get_loc(liqs.index[liq_i])
iminus = 200
iplus = 200
tdfc = tdf.iloc[max(0, liq_iloc-iminus):min(liq_iloc+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
tdfc

In [ ]:
qty_abs = tdf.qty.abs().sort_values(ascending=False)
qty_abs.head(10)

In [ ]:
# inspect biggest trades
i = 0
iloc_ = tdf.index.get_loc(qty_abs.index[i])
iminus = 400
iplus = 20
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
tdfc